In [ ]:
import numpy as np
import pandas as pd
import PIL as pl
import cv2 as cv
import pytesseract as ts
import matplotlib as plt

In [ ]:
img_pl = pl.Image.open('images/6005121460_6_page_1.png')
img_pl

In [ ]:
img_cv = cv.imread('images/6005121460_6_page_1.png')  
cv.imshow('Identity Card', img_cv)
cv.waitKey(5000)  # Wait for 5 seconds
cv.destroyAllWindows()
print(type(img_cv)) 

In [ ]:
print(type(img_pl))  



In [ ]:
text_pl = ts.image_to_string(img_pl)
print(text_pl)

In [ ]:
data = ts.image_to_data(img_pl)

In [ ]:
dataList = list(map(lambda x: x.split('\t'),data.split('\n')))

In [ ]:
df = pd.DataFrame(dataList[1:],columns=dataList[0])

In [ ]:
df.info()

In [ ]:
df.head(10)

In [ ]:
# Drop empty values and rows
df.dropna(inplace=True)

# Define columns with number values
col_int = ['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf']

# Round the 'conf' column to handle floating-point values
df['conf'] = df['conf'].astype(float).round()

# Change all columns with number values to type int
df[col_int] = df[col_int].astype(int)

# Verify the changes
print(df.dtypes)


In [ ]:
image = img_cv.copy()
level = 'word'

for l,x,y,w,h,c,t in df[['level','left','top','width','height','conf','text']].values:
    if level == 'page' and l == 1:
        cv.rectangle(image,(x,y),(x+w,y+h),(0,0,0),2)
    elif level == 'block' and l == 2:
        cv.rectangle(image,(x,y),(x+w,y+h),(255,0,0),1)
    elif level == 'paragraph' and l == 3:
        cv.rectangle(image,(x,y),(x+w,y+h),(0,255,0),1)
    elif level == 'line' and l == 4:
        cv.rectangle(image,(x,y),(x+w,y+h),(255,0,51),1)
    elif level == 'word' and l == 5:
        cv.rectangle(image,(x,y),(x+w,y+h),(0,0,255),1)
        cv.putText(image,t,(x,y),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(255,255,255),1)

# Save the image with bounding boxes
cv.imwrite("../Bounding_Box/output_with_bounding_boxes.png", image)

# Optionally, you can still display the image
cv.imshow("bounding box",image)
cv.waitKey(5000)  # Wait until a key is pressed
cv.destroyAllWindows()